In [36]:
%load_ext autoreload
%autoreload 2
import sys
import os
home_path = os.environ['HOME']
abs_dir = os.path.dirname(os.getcwd())
cwd_path = os.path.join(abs_dir, 'boias', 'bmo_br', 'real_time')
bd_path = os.path.join(abs_dir, 'boias', 'bmo_br', 'bd')

In [5]:


sys.path.append(cwd_path)
sys.path.append(bd_path)


from bmo_database import *
from bmo_adjust_data import *
import bmo_quality_control as bqc

import pandas as pd

In [23]:
import numpy as np

In [6]:
conn = connect_database_remo('PRI')

buoy_id = bmo_on(conn)

In [7]:
buoy_id

,buoy_id
0,2


In [9]:
id = buoy_id['buoy_id'][0]
id

2

In [10]:
conn = connect_database_remo('PRI')
last_date_raw = check_last_date(conn, 'bmo_br', id)

last_date_raw = last_date_raw[0][0]

In [11]:
conn = connect_database_remo('PRI')
time_interval = 24*10000 # hours interval of data to be qualified
bmo_general = get_data_table_db(conn, id, last_date_raw, 'bmo_triaxys_message', time_interval)
bmo_general[['begin', 'date', 'time', 'serial', 'buoyid', 'lat', 'lon', 'number_of_bands', 'initial_frequency',\
 'frequency_spacing', 'mean_average_direction', 'spread_direction', 'values']] = bmo_general.triaxys_message.str.split(',',12, expand=True)

bmo_general['date_time_2'] = bmo_general['date'].astype(str) + bmo_general['time']

bmo_general['date_time_2'] = pd.to_datetime(bmo_general['date_time_2'], format='%Y%m%d%H%M%S')


In [12]:
del bmo_general['begin']
del bmo_general['serial']
del bmo_general['buoyid']
del bmo_general['lat']
del bmo_general['lon']
del bmo_general['date']
del bmo_general['time']
del bmo_general['triaxys_message']

bmo_general = bmo_general.dropna()

In [13]:
bmo_general.head()

,buoy_id,id,date_time,number_of_bands,initial_frequency,frequency_spacing,mean_average_direction,spread_direction,values,date_time_2
0,2,1,2020-12-11 12:04:07,76,.060,.005,134.60,52.34,".2508E+00,194.5,31.8,.7158E+00,196.3,26.5,.735...",2020-12-11 08:29:00
1,2,2,2020-12-11 14:03:32,66,.060,.005,116.95,47.80,".1864E+00,198.3,28.4,.8430E+00,200.2,23.2,.121...",2020-12-11 10:29:00
2,2,3,2020-12-11 15:04:28,70,.060,.005,129.12,48.82,".1950E+00,194.5,37.7,.7662E+00,196.0,31.3,.889...",2020-12-11 11:29:00
3,2,4,2020-12-11 16:03:57,81,.060,.005,118.28,50.83,".3065E+00,191.2,43.8,.5087E+00,192.7,36.0,.681...",2020-12-11 12:29:00
4,2,67,2020-12-16 08:02:05,65,.075,.005,80.33,46.08,".2774E+00,173.4,66.2,.2067E+00,186.4,61.9,.392...",2020-12-16 04:29:00


In [16]:
bmo_general['values'] = bmo_general['values'].str.replace('\*\d*.*', '').str.strip().str.split(',')
bmo_general['number_of_bands'] = pd.to_numeric(bmo_general['number_of_bands'])
bmo_general['initial_frequency'] = pd.to_numeric(bmo_general['initial_frequency'])
bmo_general['frequency_spacing'] = pd.to_numeric(bmo_general['frequency_spacing'])
bmo_general['mean_average_direction'] = pd.to_numeric(bmo_general['mean_average_direction'])
bmo_general['spread_direction'] = pd.to_numeric(bmo_general['spread_direction'])

<ipython-input-16-92d3c3ea52c5>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  bmo_general['values'] = bmo_general['values'].str.replace('\*\d*.*', '').str.strip().str.split(',')


In [24]:
spacing = []
for index, row in bmo_general.iterrows():
    spacing.append(np.round(np.linspace(row['initial_frequency'], \
        row['frequency_spacing']*(row['number_of_bands'] - 1)+row['initial_frequency'], \
        row['number_of_bands']), 3))

In [25]:
energy = []
wvdir = []
spread = []
for index, row in bmo_general.iterrows():
    e = []
    w = []
    s = []
    choice = 'e'
    for i in row['values']:
        if choice == 'e':
            try:
                x = float(i)
            except:
                break
            e.append(float(i))
            choice = 'w'
        elif choice == 'w':
            try:
                x = float(i)
            except:
                break
            w.append(float(i))
            choice = 's'
        else:
            try:
                x = float(i)
            except:
                break
            s.append(float(i))
            choice = 'e'
    energy.append(e)
    wvdir.append(w)
    spread.append(s)

In [26]:
bmo_general.shape

(3505, 10)

In [27]:
len(energy)

3505

In [32]:
date_time = []
spacing_1 = []
energy_1 = []
wvdir_1 = []
spread_1 = []
mean_average_direction = []
spread_direction = []
buoy_id = []
data_id = []
for idx, value in enumerate(spacing):
    if len(value) == len(energy[idx]) and len(value) == len(wvdir[idx]) and len(value) == len(spread[idx]):
        for i in range(len(value)):
            buoy_id.append(bmo_general.iloc[idx]['buoy_id'])
            data_id.append(bmo_general.iloc[idx]['id'])
            date_time.append(bmo_general.iloc[idx]['date_time'])
            mean_average_direction.append(bmo_general.iloc[idx]['mean_average_direction'])
            spread_direction.append(bmo_general.iloc[idx]['spread_direction'])
            spacing_1.append(value[i])
            energy_1.append(energy[idx][i])
            wvdir_1.append(wvdir[idx][i])
            spread_1.append(spread[idx][i])

In [34]:
total_df = pd.DataFrame(np.array([buoy_id, data_id, date_time, mean_average_direction, spread_direction, spacing_1, energy_1, wvdir_1, spread_1]).T, \
                        columns=['buoy_id', 'data_id', 'date_time', 'mean_average_direction', 'spread_direction', 'period', 'energy', 'wvdir', 'spread'])
total_df.sort_values('date_time', inplace=True)
total_df['period'] = np.round((1/total_df['period'].astype(float)), 2)
total_df['wvdir'] = total_df['wvdir'].astype(float)
total_df['spread'] = total_df['spread'].astype(float)
total_df['mean_average_direction'] = total_df['mean_average_direction'].astype(float)
total_df['spread_direction'] = total_df['spread_direction'].astype(float)

In [41]:
total_df['energy'] = total_df['energy'].astype(float)
total_df['buoy_id'] = total_df['buoy_id'].astype(int)
total_df['data_id'] = total_df['data_id'].astype(int)

In [38]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150716 entries, 0 to 150715
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   buoy_id                 150716 non-null  object        
 1   data_id                 150716 non-null  object        
 2   date_time               150716 non-null  datetime64[ns]
 3   mean_average_direction  150716 non-null  float64       
 4   spread_direction        150716 non-null  float64       
 5   period                  150716 non-null  float64       
 6   energy                  150716 non-null  object        
 7   wvdir                   150716 non-null  float64       
 8   spread                  150716 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 11.5+ MB


In [40]:
insert_triaxys_data(total_df)

inserted new data
